In [10]:
sentences = [
    'The RTX3080 is a marvel to behold.',
    'This model"s price has decreased compared to previous models.',
    'The temperature of this model is a bit hot.',
    'The cooling is honestly terrible.',
    'The next installment in the RTX-series is one of a kind.',
    'The RTX 3080 is a beast.'
]

In [56]:
import nltk
import os
from nltk.corpus import stopwords

stops = stopwords.words('english')
stops

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [57]:
global dtn
global adjectives
adjectives = []
global all_sentnc
all_sentnc = []

def processing(id, content):
    global dtn
    global adjectives
    global all_sentnc

    sentnc_list = []
    sentnc_list_NN = []
    sentnc_list_JJ = []

    sentence_info = []
    sentence_info.append(id)
    sentence_info.append(content)

    all_sentnc.append(sentence_info)
    sentence_info = []

    tokenized = nltk.word_tokenize(content)
    tagged_list = nltk.pos_tag(tokenized)

    for entry in tagged_list:
        if "JJ" in entry:
            sentnc_list_JJ.append(entry[0])
        elif "NN" in entry or "NNS" in entry or "NNP" in entry or "NNPS" in entry:
            sentnc_list_NN.append(entry[0])

    sentnc_list.append(sentnc_list_NN)
    sentnc_list.append(sentnc_list_JJ)

    adjectives.append(sentnc_list)

In [58]:
for i in range(len(sentences)):
    processing(i, sentences[i])

In [59]:
def getSentences():
    return all_sentnc

In [60]:
def getAspectTermDict():
    return adjectives

In [61]:
aspect_term = getAspectTermDict()
sentence_list = getSentences()

aspect_terms = []

In [62]:
def detect_terms(temp, sentence, nnlist):
    aspect_term = []
    for relation in temp:
        if relation[0] == 'amod':
            aspect_term.append(relation[2].split('-')[0])
        if (relation[0]=='acomp' or relation[0]=='xcomp' or relation[0]=='nmod' or relation[0]=='dobj'
		or relation[0]=='nsubj' or relation[0]=='nsubjpass' or relation[0]=='xcomp' 
		or relation[0]=='pobj' or relation[0]=='abbrev'):
		        aspect_term.append(relation[4].split('-')[0])

    aspect_term = list(set(aspect_term))
    for term in aspect_term:
        if term not in nnlist:
            aspect_term.remove(temp)
    for relation in temp:
        if relation[0] == 'conj' and relation[2].split('-')[0] in aspect_term:
            aspect_term.append(relation[4].split('-')[0])
    aspect_term = list(set(aspect_term))
    return aspect_term

def findvblist(sentence):
    sent = ""
    token = sentence.split(" ")
    for word in token:
        if word not in stops:
            sent = sent+word
            sent = sent+" "
    sent.strip()
    token = nltk.pos_tag(sent)
    tagged_list = nltk.pos_tag(token)

    vblist = []
    for tags in tagged_list:
        if (tags[1]=='VB' or tags[1]=='VBD' or tags[1]=='VBG' or tags[1]=='VBN' or tags[1]=='VBP'
		or tags[1]=='VBZ'):
            vblist.append(tags[0])
    return vblist

def detect_quality(term, data, JJ_list, mydict, depth, key):
    if depth == 2:
        mydict[key] = list(set(mydict[key]))
        return mydict
    for relation in data:
        if len(relation) >= 5:
            if relation[2].split('-')[0] == term:
                if relation[4].split('-')[0] in JJ_list:
                    mydict[key].append(relation[4].split('-')[0])
                detect_quality(relation[4].split('-')[0], data, JJ_list, mydict, depth+1, key)
            elif relation[4].split('-')[0] == term:
                if relation[2].split('-')[0] in JJ_list:
                    mydict[key].append(relation[2].split('-')[0])
                detect_quality(relation[2].split('-')[0], data, JJ_list, mydict, depth+1, key)
    mydict[key] = list(set(mydict[key]))
    return mydict

for i in range(len(sentence_list)):
    sentence = sentence_list[i][1]
    pos_list = aspect_term[i]
    temp = []
    terms = []
    os.popen("echo '" + sentence + "' > ./stanford-parser-full/data/sentence.txt")
    parser_out = os.popen("./stanford-parser-full/lexparser.sh ./stanford-parser-full/data/sentence.txt")
    for relation in parser_out:
        relation = nltk.word_tokenize(relation)
        if len(relation) != 0:
            temp.append(relation)
    x = detect_terms(temp, sentence, pos_list[0])
    JJ_list = pos_list[1]
    if len(JJ_list) == 0:
        JJ_list = findvblist(sentence)
    for term in x:
        mydict = {}
        mydict[term] = []
        terms.append(detect_quality(term, temp, JJ_list, mydict, 0, term))
    aspect_terms.append(terms)
aspect_terms
    

TypeError: tokens: expected a list of strings, got a string